In [3]:
import kagglehub
import shutil

In [2]:
import kagglehub

path = kagglehub.dataset_download("muratkokludataset/rice-image-dataset")
print("Dataset downloaded to:", path)


100%|██████████| 219M/219M [00:04<00:00, 56.6MB/s] 

Extracting files...


Dataset downloaded to: C:\Users\felip\.cache\kagglehub\datasets\muratkokludataset\rice-image-dataset\versions\1


In [6]:
dst = r"C:\Users\felip\OneDrive - Rice University\CNN"
shutil.copytree(path, dst, dirs_exist_ok=True)

'C:\\Users\\felip\\OneDrive - Rice University\\CNN'

In [8]:
import os

data_dir = r"C:\Users\felip\OneDrive - Rice University\CNN\Rice_Image_Dataset"
classes = [d.name for d in os.scandir(data_dir) if d.is_dir()]
for cls in classes:
    print(cls, ":", len(os.listdir(os.path.join(data_dir, cls))), "images")


Arborio : 15000 images
Basmati : 15000 images
Ipsala : 15000 images
Jasmine : 15000 images
Karacadag : 15000 images
